# Inspection of Intermediate Results

In iterative algorithms like DSEA, IBU, and RUN you may want to inspect intermediate results. This tutorial assumes you already know the other notebook at `doc/01-getting-started.ipynb`.

In [ ]:
using CherenkovDeconvolution
using ScikitLearn, MLDataUtils, Random
using Discretizers: encode, CategoricalDiscretizer

# load the example data, encode labels with integers
X, y_labels, _ = load_iris()
y = encode(CategoricalDiscretizer(y_labels), y_labels)

# split the data into training and observed data sets
Random.seed!(42) # make split reproducible
(X_train, y_train), (X_data, y_data) = splitobs(shuffleobs((X', y), obsdim = 1), obsdim = 1)

# prepare the arguments for all deconvolution methods
@sk_import naive_bayes : GaussianNB # naive Bayes for DSEA
binning = TreeBinning(3) # up to 3 clusters for IBU & RUN

## Inspection in DSEA

Inspection is realized through the keyword argument `inspect` available in all iterative algorithms. This argument accept a `Function` object, which will be called in each iteration of the deconvolution algorithm. Depending on the algorithm, this `Function` object has to have different signatures.

For DSEA, the `inspect` function has to have the following signature:

    (f_k::Vector, k::Int, chi2s::Float64, alpha::Float64) -> Any

You do not have to stick to the parameter names (`f_k`, `k`, etc) and you do not even have to specify the types of the arguments explicitly. However, these are the types that the parameters will have, so do not expect anything else. The return value of the `inspect` function is never used, so you can return any value, including `nothing`.

The first parameter of `inspect`, `f_k`, refers to the intermediate result of the `k`-th iteration. `chi2s` is the Chi-Square distance between `f_k` and the previous estimate. This distance may be used to check convergence. `alpha` is the step size used in the `k`-th iteration of DSEA.

In [ ]:
# we want to store all inspection results in a single DataFrame
using DataFrames
df = DataFrame(f=Vector{Float64}[], k=Int[], chi2s=Float64[], a=Float64[]) # empty frame with fixed types

# set up the inspection function
inspect_function = (f, k, chi2s, a) -> push!(df, [f, k, chi2s, a]) # store results in df

# provide inspect_function as a keyword argument to DSEA, make 3 iterations and return the final result
dsea = DSEA(GaussianNB(); K=3, inspect=inspect_function)
deconvolve(dsea, X_data, X_train, y_train)

In [ ]:
# let's have a look at the DataFrame - beautiful, isn't it?
df

## Inspection in RUN and IBU

IBU is inspected just like DSEA. The `inspect` function of RUN, however, has a different signature:

    (f_k::Array, k::Int, ldiff::Float64, tau::Float64) -> Any

Here, `ldiff` stores the difference in the likelihood loss of RUN between two iterations. `tau` is the regularization parameter chosen in the `k`-th iteration.

In [ ]:
# set up an inspection function for IBU - do not store anything, just print
inspect_ibu = (f, k, chi2s, alpha) -> println("This is iteration $k with chi2s=$chi2s")

ibu = IBU(binning; inspect=inspect_ibu) # by default, IBU stops after 3 iterations
deconvolve(ibu, X_data, X_train, y_train)